In [5]:
#DATA INTEGRITY CHECK

import pandas as pd

# --- Load (fastparquet) ---
path = "network_data.parquet"
df = pd.read_parquet(path, engine="fastparquet")

# --- Basic snapshot ---
print("Shape:", df.shape)
print("\nColumns:", list(df.columns))
print("\nDtypes:\n", df.dtypes)
print("\nHead:\n", df.head(3))

# --- Required columns check ---
required = {"start_station_id", "end_station_id", "started_at"}
missing = sorted(required - set(df.columns))
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# --- Null / missingness ---
print("\nMissingness (%):")
print((df.isna().mean() * 100).sort_values(ascending=False).round(3))

# --- Basic validity checks ---
# 1) Empty / whitespace station IDs
def _blank(x):
    return x.isna() | (x.astype(str).str.strip() == "")

blank_start = _blank(df["start_station_id"]).mean()
blank_end   = _blank(df["end_station_id"]).mean()
print(f"\nBlank start_station_id (%): {blank_start*100:.4f}")
print(f"Blank end_station_id   (%): {blank_end*100:.4f}")

# 2) Ensure started_at is datetime
if not pd.api.types.is_datetime64_any_dtype(df["started_at"]):
    df["started_at"] = pd.to_datetime(df["started_at"], errors="coerce", utc=False)

bad_time = df["started_at"].isna().mean()
print(f"\nUnparseable started_at (%): {bad_time*100:.4f}")

# 3) Duplicates (exact row duplicates)
dup_rate = df.duplicated().mean()
print(f"\nExact duplicate rows (%): {dup_rate*100:.4f}")

# 4) Self-loops (start == end)
self_loop_rate = (df["start_station_id"].astype(str) == df["end_station_id"].astype(str)).mean()
print(f"\nSelf-loop trips (%): {self_loop_rate*100:.4f}")

# --- Cardinalities / coverage ---
n_start = df["start_station_id"].nunique(dropna=True)
n_end   = df["end_station_id"].nunique(dropna=True)
n_nodes = pd.Index(df["start_station_id"].dropna().astype(str)).union(
          pd.Index(df["end_station_id"].dropna().astype(str))).nunique()

print("\nUnique counts:")
print("  Unique start stations:", n_start)
print("  Unique end stations  :", n_end)
print("  Unique stations total:", n_nodes)

# --- Time range & basic distribution ---
tmin, tmax = df["started_at"].min(), df["started_at"].max()
print("\nTime range:")
print("  min:", tmin)
print("  max:", tmax)

# --- Station name consistency (optional, if columns exist) ---
# Checks whether an ID maps to multiple names (data hygiene)
for id_col, name_col in [
    ("start_station_id", "start_station_name"),
    ("end_station_id", "end_station_name"),
]:
    if id_col in df.columns and name_col in df.columns:
        tmp = df[[id_col, name_col]].dropna()
        multi_name = (tmp.groupby(tmp[id_col].astype(str))[name_col].nunique() > 1).mean()
        print(f"\n{ id_col } -> multiple { name_col } mappings (% of IDs): {multi_name*100:.4f}")

# --- Tripduration sanity (optional) ---
if "tripduration" in df.columns:
    non_null = df["tripduration"].notna().mean()
    print(f"\ntripduration non-null (%): {non_null*100:.4f}")
    if non_null > 0:
        # show a few stats if it's populated
        td = pd.to_numeric(df["tripduration"], errors="coerce")
        print("tripduration stats (sec):")
        print(td.describe(percentiles=[0.01, 0.5, 0.99]).round(3))

issues = []
if blank_start > 0: issues.append("blank start_station_id")
if blank_end > 0: issues.append("blank end_station_id")
if bad_time > 0: issues.append("unparseable started_at")

print("\nINTEGRITY VERDICT:", "OK ✅" if not issues else f"CHECK ⚠️ ({', '.join(issues)})")


Shape: (4702180, 6)

Columns: ['start_station_id', 'end_station_id', 'started_at', 'start_station_name', 'end_station_name', 'tripduration']

Dtypes:
 start_station_id              object
end_station_id                object
started_at            datetime64[ns]
start_station_name            object
end_station_name              object
tripduration                 float64
dtype: object

Head:
   start_station_id end_station_id              started_at  \
0           M32085         S32023 2024-08-01 19:57:49.371   
1           K32017         D32035 2024-08-01 19:57:49.672   
2           C32077         C32005 2024-08-01 19:57:49.879   

             start_station_name            end_station_name  tripduration  
0     Mass Ave/Lafayette Square                  30 Dane St           NaN  
1     Harvard St and Stedman St  Harvard Ave at Brainerd Rd           NaN  
2  Columbus Ave at W. Canton St   Washington St at Lenox St           NaN  

Missingness (%):
tripduration          100.0
start_stat

In [5]:
import pandas as pd
import numpy as np
import networkx as nx
import warnings
# Ignore all warnings (global)
warnings.filterwarnings("ignore")


PATH = "network_data.parquet"
df = pd.read_parquet(PATH, engine="fastparquet")

df = df.drop_duplicates().reset_index(drop=True)

# Ensure datetime
df["started_at"] = pd.to_datetime(df["started_at"], errors="coerce")
df = df.dropna(subset=["start_station_id", "end_station_id", "started_at"])

# Hour bin for aggregation (kept for future extensions, not strictly needed for season graphs)
df["hour"] = df["started_at"].dt.floor("H")

# Season label (meteorological seasons)
def season_from_month(m: int) -> str:
    if m in (12, 1, 2):  return "Winter"
    if m in (3, 4, 5):   return "Spring"
    if m in (6, 7, 8):   return "Summer"
    return "Fall"

df["season"] = df["started_at"].dt.month.map(season_from_month)

start_lookup = (
    df[["start_station_id", "start_station_name"]]
    .dropna()
    .groupby("start_station_id")["start_station_name"]
    .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else s.iloc[0])
)
end_lookup = (
    df[["end_station_id", "end_station_name"]]
    .dropna()
    .groupby("end_station_id")["end_station_name"]
    .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else s.iloc[0])
)

# One station-name dict for labeling
name_lookup = start_lookup.combine_first(end_lookup).to_dict()

# ----------------------------
# 2) Build edge lists (overall + by season)
# ----------------------------
def make_edges(frame: pd.DataFrame) -> pd.DataFrame:
    # Weighted directed edges: count trips as weight
    edges = (
        frame.groupby(["start_station_id", "end_station_id"])
             .size()
             .reset_index(name="weight")
    )
    return edges

edges_all = make_edges(df)
edges_by_season = {s: make_edges(df[df["season"] == s]) for s in ["Winter", "Spring", "Summer", "Fall"]}

# ----------------------------
# 3) Graph build + centralities
# ----------------------------
def build_graph(edges: pd.DataFrame) -> nx.DiGraph:
    G = nx.DiGraph()
    for u, v, w in edges.itertuples(index=False):
        G.add_edge(u, v, weight=float(w))
    return G

def centralities(G: nx.DiGraph, drop_self_loops_for_betweenness: bool = True) -> pd.DataFrame:
    # Strength = weighted degree
    out_strength = dict(G.out_degree(weight="weight"))
    in_strength  = dict(G.in_degree(weight="weight"))

    # Weighted PageRank
    pr = nx.pagerank(G, weight="weight")

    # Betweenness: interpret "cost" as inverse of flow weight (stronger flow = shorter distance)
    H = G.copy()
    if drop_self_loops_for_betweenness:
        H.remove_edges_from(nx.selfloop_edges(H))

    for u, v, d in H.edges(data=True):
        w = d.get("weight", 1.0)
        d["distance"] = 1.0 / (w + 1e-12)

    btw = nx.betweenness_centrality(H, weight="distance", normalized=True)

    # Combine
    nodes = sorted(set(G.nodes()))
    out_s = pd.Series(out_strength, name="out_strength").reindex(nodes).fillna(0.0)
    in_s  = pd.Series(in_strength,  name="in_strength").reindex(nodes).fillna(0.0)
    pr_s  = pd.Series(pr,           name="pagerank").reindex(nodes).fillna(0.0)
    btw_s = pd.Series(btw,          name="betweenness").reindex(nodes).fillna(0.0)

    out = pd.concat([out_s, in_s, pr_s, btw_s], axis=1).reset_index(names="station_id")
    out["station_name"] = out["station_id"].map(name_lookup)
    return out

# ----------------------------
# 4) Consensus top-k via rank aggregation
# ----------------------------
def add_consensus_rank(metrics: pd.DataFrame) -> pd.DataFrame:
    m = metrics.copy()

    # Higher is better → rank descending
    for col in ["out_strength", "betweenness", "pagerank"]:
        m[f"rank_{col}"] = m[col].rank(ascending=False, method="average")

    # Borda-style: lower total rank = better
    m["consensus_rank_sum"] = m[["rank_out_strength", "rank_betweenness", "rank_pagerank"]].sum(axis=1)
    m["consensus_rank"] = m["consensus_rank_sum"].rank(ascending=True, method="dense")

    # Handy plotting score
    m["consensus_score"] = 1.0 / (m["consensus_rank_sum"] + 1e-12)

    return m.sort_values(["consensus_rank_sum"]).reset_index(drop=True)

# Compute metrics
G_all = build_graph(edges_all)
metrics_all = add_consensus_rank(centralities(G_all))

metrics_season = {}
for s, e in edges_by_season.items():
    Gs = build_graph(e)
    metrics_season[s] = add_consensus_rank(centralities(Gs))

# ----------------------------
# 5) Top-20 lists + stability (Jaccard overlap)
# ----------------------------
def top_k_ids(metrics: pd.DataFrame, k: int = 20, by: str = "consensus_rank_sum"):
    return list(metrics.sort_values(by).head(k)["station_id"])

def jaccard(a, b) -> float:
    A, B = set(a), set(b)
    return len(A & B) / max(1, len(A | B))

seasons = ["Winter", "Spring", "Summer", "Fall"]
top20_by_season = {s: top_k_ids(metrics_season[s], 20) for s in seasons}

stab = pd.DataFrame(index=seasons, columns=seasons, dtype=float)
for i in seasons:
    for j in seasons:
        stab.loc[i, j] = jaccard(top20_by_season[i], top20_by_season[j])

def round_numeric(df_in: pd.DataFrame, decimals: int = 3) -> pd.DataFrame:
    out = df_in.copy()
    num_cols = out.select_dtypes(include=[np.number]).columns
    out[num_cols] = out[num_cols].round(decimals)
    return out

metrics_all_r = round_numeric(metrics_all, 3)
metrics_season_r = {s: round_numeric(m, 3) for s, m in metrics_season.items()}
stab_r = stab.round(3)

metrics_all_r.to_csv("network_metrics_all.csv", index=False)
for s in seasons:
    metrics_season_r[s].to_csv(f"network_metrics_{s.lower()}.csv", index=False)

stab_r.to_csv("top20_stability_jaccard.csv")

top20_all = metrics_all_r.sort_values("consensus_rank_sum").head(20)
top20_all.to_csv("top20_consensus_all.csv", index=False)

for s in seasons:
    top20 = metrics_season_r[s].sort_values("consensus_rank_sum").head(20)
    top20.to_csv(f"top20_consensus_{s.lower()}.csv", index=False)

print("Done.")
print("Saved: network_metrics_all.csv, network_metrics_<season>.csv, top20_consensus_*.csv, top20_stability_jaccard.csv")
print("\nTop-20 stability matrix (Jaccard overlap, rounded to 3 decimals):\n", stab_r)


Done.
Saved: network_metrics_all.csv, network_metrics_<season>.csv, top20_consensus_*.csv, top20_stability_jaccard.csv

Top-20 stability matrix (Jaccard overlap, rounded to 3 decimals):
         Winter  Spring  Summer   Fall
Winter   1.000   0.667   0.481  0.538
Spring   0.667   1.000   0.739  0.667
Summer   0.481   0.739   1.000  0.667
Fall     0.538   0.667   0.667  1.000


In [6]:
import pandas as pd
import numpy as np
import re
import requests
import folium

# --- Load your parquet (fastparquet) ---
PATH = "network_data.parquet"
df = pd.read_parquet(PATH, engine="fastparquet").drop_duplicates()

df["start_station_id"] = df["start_station_id"].astype(str).str.strip()
df["end_station_id"]   = df["end_station_id"].astype(str).str.strip()

# Build canonical names from your parquet (mode name per station_id)
start_lookup = (
    df[["start_station_id", "start_station_name"]]
    .dropna()
    .groupby("start_station_id")["start_station_name"]
    .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else s.iloc[0])
)
end_lookup = (
    df[["end_station_id", "end_station_name"]]
    .dropna()
    .groupby("end_station_id")["end_station_name"]
    .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else s.iloc[0])
)

name_lookup = start_lookup.combine_first(end_lookup).to_dict()

metrics_all_r = pd.read_csv("network_metrics_all.csv")
metrics_all_r["station_id"] = metrics_all_r["station_id"].astype(str).str.strip()

# Ensure station_name exists
if "station_name" not in metrics_all_r.columns:
    metrics_all_r["station_name"] = metrics_all_r["station_id"].map(name_lookup)

GBFS_DISCOVERY = "https://gbfs.bluebikes.com/gbfs/gbfs.json"
gbfs = requests.get(GBFS_DISCOVERY, timeout=30).json()
feeds = gbfs["data"]["en"]["feeds"]
station_info_url = next(f["url"] for f in feeds if f["name"] == "station_information")

station_info = requests.get(station_info_url, timeout=30).json()
gbfs_st = pd.DataFrame(station_info["data"]["stations"])

gbfs_st = gbfs_st.rename(columns={"name": "gbfs_station_name"}).copy()
gbfs_st["gbfs_station_name"] = gbfs_st["gbfs_station_name"].astype(str)

gbfs_st["lat"] = pd.to_numeric(gbfs_st["lat"], errors="coerce")
gbfs_st["lon"] = pd.to_numeric(gbfs_st["lon"], errors="coerce")

def norm_name(s: str) -> str:
    s = s.lower().strip()
    s = s.replace("&", " and ")
    s = re.sub(r"[^a-z0-9\s]", " ", s)     # remove punctuation
    s = re.sub(r"\s+", " ", s).strip()    # collapse whitespace
    # common cleanups (optional)
    s = s.replace(" at ", " ")            # "X at Y" vs "X/Y" style drift
    return s

# normalized tables
parquet_names = (
    pd.Series(name_lookup, name="station_name")
      .reset_index()
      .rename(columns={"index": "station_id"})
)
parquet_names["norm"] = parquet_names["station_name"].astype(str).map(norm_name)

gbfs_st["norm"] = gbfs_st["gbfs_station_name"].map(norm_name)

# Use exact normalized match first
gbfs_by_norm = gbfs_st.dropna(subset=["lat","lon"]).drop_duplicates("norm").set_index("norm")[["gbfs_station_name","lat","lon"]]
parquet_names = parquet_names.join(gbfs_by_norm, on="norm", how="left")

# Report match rate
match_rate = parquet_names["lat"].notna().mean() * 100
print(f"Exact normalized name match rate: {match_rate:.2f}%")

unmatched = parquet_names[parquet_names["lat"].isna()][["station_id","station_name"]].head(10)
if len(unmatched):
    print("\nExample unmatched stations (first 10):")
    print(unmatched.to_string(index=False))

# Build station_id -> lat/lon mapping
id_to_latlon = parquet_names.set_index("station_id")[["lat","lon","gbfs_station_name"]]

metrics_geo = metrics_all_r.merge(
    id_to_latlon.reset_index(),
    on="station_id",
    how="left"
)

plot_df = metrics_geo.dropna(subset=["lat","lon"]).copy()
if plot_df.empty:
    raise ValueError(
        "No stations matched to lat/lon. "
        "This suggests station names differ too much; we can add fuzzy matching."
    )

center_lat, center_lon = float(plot_df["lat"].mean()), float(plot_df["lon"].mean())
m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="CartoDB positron")

# Marker radius based on consensus_score (log scaled)
cs = pd.to_numeric(plot_df["consensus_score"], errors="coerce").fillna(0).values
radii = (np.log1p(cs * 1e6) + 1) * 2.5

for (_, row), r in zip(plot_df.iterrows(), radii):
    nm = row.get("gbfs_station_name") if pd.notna(row.get("gbfs_station_name")) else row.get("station_name")
    tooltip = (
        f"{nm}<br>"
        f"station_id={row['station_id']}<br>"
        f"consensus_score={float(row['consensus_score']):.3f}<br>"
        f"pagerank={float(row['pagerank']):.3f}, betweenness={float(row['betweenness']):.3f}<br>"
        f"out_strength={float(row['out_strength']):.3f}, in_strength={float(row['in_strength']):.3f}"
    )
    folium.CircleMarker(
        location=[float(row["lat"]), float(row["lon"])],
        radius=float(r),
        fill=True,
        fill_opacity=0.70,
        opacity=0.80,
        tooltip=folium.Tooltip(tooltip, sticky=True),
    ).add_to(m)

OUT_HTML = "boston_bluebikes_network_map.html"
m.save(OUT_HTML)
print(f"\nSaved interactive map: {OUT_HTML}")
print(f"Stations plotted: {len(plot_df)} / {len(metrics_geo)} ({len(plot_df)/len(metrics_geo)*100:.2f}%)")


Exact normalized name match rate: 95.98%

Example unmatched stations (first 10):
station_id                     station_name
      <NA>                             <NA>
    A32003    B.U. Central - 725 Comm. Ave.
    A32037     Washington St at Egremont Rd
    A32043     Western Ave at Richardson St
    B32017 Dudley Square - Bolling Building
    B32060            700 Commonwealth Ave.
    BCBS01                     BCBS Hingham
    C32049       Thetford Ave at Norfolk St
    C32063               Mass Ave T Station
    C32065          Adams St at Lonsdale St

Saved interactive map: boston_bluebikes_network_map.html
Stations plotted: 549 / 572 (95.98%)


In [9]:
import numpy as np
import folium

# --- PARAMETERS ---
TOP_N = 120          
MIN_R = 3
MAX_R = 14

# Keep only top-N by importance
plot_top = plot_df.sort_values("consensus_score", ascending=False).head(TOP_N).copy()

# Robust scaling: quantiles -> [MIN_R, MAX_R]
scores = plot_top["consensus_score"].astype(float).values
lo, hi = np.quantile(scores, 0.05), np.quantile(scores, 0.95)
scaled = (np.clip(scores, lo, hi) - lo) / (hi - lo + 1e-12)
radii = MIN_R + scaled * (MAX_R - MIN_R)

# Center map
center_lat, center_lon = float(plot_top["lat"].mean()), float(plot_top["lon"].mean())
m1 = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="CartoDB positron")

for (_, row), r in zip(plot_top.iterrows(), radii):
    nm = row.get("gbfs_station_name") if "gbfs_station_name" in row and not pd.isna(row.get("gbfs_station_name")) else row.get("station_name")
    tooltip = (
        f"{nm}<br>"
        f"consensus_score={float(row['consensus_score']):.3f}<br>"
        f"pagerank={float(row['pagerank']):.3f}, betweenness={float(row['betweenness']):.3f}<br>"
        f"out_strength={float(row['out_strength']):.3f}, in_strength={float(row['in_strength']):.3f}"
    )

    # "Nicer" marker: circle + border and a bit more contrast
    folium.CircleMarker(
        location=[float(row["lat"]), float(row["lon"])],
        radius=float(r),
        weight=2,            # border thickness
        fill=True,
        fill_opacity=0.75,
        opacity=0.9,
        tooltip=folium.Tooltip(tooltip, sticky=True),
    ).add_to(m1)

OUT1 = "boston_network_topN_markers.html"
m1.save(OUT1)
print(f"Saved: {OUT1} (Top {TOP_N} stations)")


Saved: boston_network_topN_markers.html (Top 120 stations)


In [7]:
import folium
from folium.plugins import HeatMap
import numpy as np

# --- PARAMETERS ---
TOP_N = 250          
WEIGHT_POWER = 0.5   # <1 compresses extremes; >1 emphasizes top hubs
RADIUS = 18
BLUR = 22
MAX_ZOOM = 13

plot_heat = plot_df.sort_values("consensus_score", ascending=False).head(TOP_N).copy()

# Create heat weights from consensus_score
w = plot_heat["consensus_score"].astype(float).values
w = np.power(w / (w.max() + 1e-12), WEIGHT_POWER)  # normalize then shape distribution

heat_data = list(zip(plot_heat["lat"].astype(float), plot_heat["lon"].astype(float), w))

center_lat, center_lon = float(plot_heat["lat"].mean()), float(plot_heat["lon"].mean())
m2 = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="CartoDB positron")

HeatMap(
    heat_data,
    radius=RADIUS,
    blur=BLUR,
    max_zoom=MAX_ZOOM,
).add_to(m2)

OUT2 = "boston_network_heatmap.html"
m2.save(OUT2)
print(f"Saved: {OUT2} (Top {TOP_N} stations)")


Saved: boston_network_heatmap.html (Top 250 stations)


In [8]:
import numpy as np
import folium
from folium.plugins import HeatMap

# plot_df must contain lat, lon, consensus_score, out_strength
# If out_strength is huge, we normalize; Folium heatmap expects relative weights.

def make_heatmap(df_in, weight_col, out_html,
                 top_n=300, weight_power=0.5, radius=18, blur=22, max_zoom=13):
    d = df_in.dropna(subset=["lat", "lon", weight_col]).copy()
    d = d.sort_values(weight_col, ascending=False).head(top_n)

    w = d[weight_col].astype(float).values
    # normalize then shape distribution so it isn't dominated by a few stations
    w = np.power(w / (w.max() + 1e-12), weight_power)

    heat_data = list(zip(d["lat"].astype(float), d["lon"].astype(float), w))

    center_lat, center_lon = float(d["lat"].mean()), float(d["lon"].mean())
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="CartoDB positron")

    HeatMap(heat_data, radius=radius, blur=blur, max_zoom=max_zoom).add_to(m)
    m.save(out_html)
    print(f"Saved: {out_html} (heatmap of {weight_col}, top_n={top_n})")

# 1) Importance density (consensus_score)
make_heatmap(
    plot_df,
    weight_col="consensus_score",
    out_html="boston_heat_importance.html",
    top_n=300,
    weight_power=0.5
)

# 2) Demand density (out_strength)
make_heatmap(
    plot_df,
    weight_col="out_strength",
    out_html="boston_heat_demand.html",
    top_n=300,
    weight_power=0.5
)


Saved: boston_heat_importance.html (heatmap of consensus_score, top_n=300)
Saved: boston_heat_demand.html (heatmap of out_strength, top_n=300)


In [9]:
import numpy as np
import pandas as pd
import folium

def marker_map(df_in, weight_col, out_html,
               top_n=120, min_r=3, max_r=14):
    d = df_in.dropna(subset=["lat", "lon", weight_col]).copy()
    d = d.sort_values(weight_col, ascending=False).head(top_n)

    # quantile scaling so sizes spread nicely
    w = d[weight_col].astype(float).values
    lo, hi = np.quantile(w, 0.05), np.quantile(w, 0.95)
    scaled = (np.clip(w, lo, hi) - lo) / (hi - lo + 1e-12)
    radii = min_r + scaled * (max_r - min_r)

    center_lat, center_lon = float(d["lat"].mean()), float(d["lon"].mean())
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="CartoDB positron")

    for (_, row), r in zip(d.iterrows(), radii):
        nm = row.get("gbfs_station_name")
        if pd.isna(nm) or nm is None:
            nm = row.get("station_name", row.get("station_id"))

        tooltip = (
            f"{nm}<br>"
            f"{weight_col}={float(row[weight_col]):.3f}<br>"
            f"pagerank={float(row.get('pagerank', 0)):.3f}, betweenness={float(row.get('betweenness', 0)):.3f}<br>"
            f"out_strength={float(row.get('out_strength', 0)):.3f}, in_strength={float(row.get('in_strength', 0)):.3f}"
        )

        folium.CircleMarker(
            location=[float(row["lat"]), float(row["lon"])],
            radius=float(r),
            weight=2,            # border thickness
            fill=True,
            fill_opacity=0.75,
            opacity=0.9,
            tooltip=folium.Tooltip(tooltip, sticky=True),
        ).add_to(m)

    m.save(out_html)
    print(f"Saved: {out_html} (marker size = {weight_col}, top_n={top_n})")

# --- Importance map ---
marker_map(
    plot_df,
    weight_col="consensus_score",
    out_html="boston_markers_importance.html",
    top_n=120
)

# --- Demand map ---
marker_map(
    plot_df,
    weight_col="out_strength",
    out_html="boston_markers_demand.html",
    top_n=120
)


Saved: boston_markers_importance.html (marker size = consensus_score, top_n=120)
Saved: boston_markers_demand.html (marker size = out_strength, top_n=120)


In [10]:
import numpy as np
import pandas as pd
import folium

def add_legend(m, title, labels, colors):
    # Simple HTML legend box
    items = "".join(
        f"<div style='display:flex; align-items:center; margin:2px 0;'>"
        f"<span style='width:14px; height:14px; background:{c}; display:inline-block; "
        f"margin-right:8px; border:1px solid #666;'></span>"
        f"<span style='font-size:12px;'>{lab}</span></div>"
        for lab, c in zip(labels, colors)
    )
    html = f"""
    <div style="
        position: fixed; bottom: 25px; left: 25px; z-index: 9999;
        background: rgba(255,255,255,0.92); padding: 10px 12px;
        border: 1px solid #999; border-radius: 8px; box-shadow: 0 1px 8px rgba(0,0,0,0.2);
        ">
        <div style="font-size:13px; font-weight:600; margin-bottom:6px;">{title}</div>
        {items}
    </div>
    """
    m.get_root().html.add_child(folium.Element(html))

def marker_map_emphasized(
    df_in,
    weight_col,
    out_html,
    top_n=120,
    min_r=3,
    max_r=22,
    gamma=2.0,         # >1 emphasizes top stations more
    n_bins=5,          # quantile bins for color
    show_edges=False
):
    d = df_in.dropna(subset=["lat", "lon", weight_col]).copy()
    d[weight_col] = pd.to_numeric(d[weight_col], errors="coerce")
    d = d.dropna(subset=[weight_col])

    d = d.sort_values(weight_col, ascending=False).head(top_n).copy()

    # ----- Size scaling -----
    w = d[weight_col].astype(float).values
    # robust range
    lo, hi = np.quantile(w, 0.05), np.quantile(w, 0.95)
    x = (np.clip(w, lo, hi) - lo) / (hi - lo + 1e-12)  # [0,1]
    x = np.power(x, gamma)                              # emphasize top
    radii = min_r + x * (max_r - min_r)

    # ----- Color bins (quantiles) -----
    # Create bin edges from quantiles of the selected top_n
    qs = np.quantile(w, np.linspace(0, 1, n_bins + 1))
    # ensure strictly increasing edges (rare ties can cause duplicates)
    qs = np.unique(qs)
    if len(qs) < 3:
        # fallback if values are too tied
        qs = np.array([w.min(), np.median(w), w.max()])

    # Define a simple light->dark palette (you can change these)
    colors = ["#cfe8ff", "#9ecae1", "#6baed6", "#3182bd", "#08519c"]
    colors = colors[-(len(qs)-1):]  # match number of intervals

    def color_for(val):
        # find interval index
        idx = np.searchsorted(qs, val, side="right") - 1
        idx = max(0, min(idx, len(qs) - 2))
        return colors[idx]

    # Legend labels: show ranges
    labels = []
    for i in range(len(qs)-1):
        labels.append(f"{qs[i]:.3f} – {qs[i+1]:.3f}")

    # ----- Build map -----
    center_lat, center_lon = float(d["lat"].mean()), float(d["lon"].mean())
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="CartoDB positron")

    for (_, row), r in zip(d.iterrows(), radii):
        nm = row.get("gbfs_station_name")
        if pd.isna(nm) or nm is None:
            nm = row.get("station_name", row.get("station_id"))

        val = float(row[weight_col])
        tooltip = (
            f"{nm}<br>"
            f"{weight_col}={val:.3f}<br>"
            f"pagerank={float(row.get('pagerank', 0)):.3f}, betweenness={float(row.get('betweenness', 0)):.3f}<br>"
            f"out_strength={float(row.get('out_strength', 0)):.3f}, in_strength={float(row.get('in_strength', 0)):.3f}"
        )

        folium.CircleMarker(
            location=[float(row["lat"]), float(row["lon"])],
            radius=float(r),
            color="#333333",
            weight=1,
            fill=True,
            fill_color=color_for(val),
            fill_opacity=0.78,
            opacity=0.9,
            tooltip=folium.Tooltip(tooltip, sticky=True),
        ).add_to(m)

    # Add legend
    add_legend(
        m,
        title=f"{weight_col} bins (Top {top_n})",
        labels=labels,
        colors=colors
    )

    m.save(out_html)
    print(f"Saved: {out_html}  | size~{weight_col} (gamma={gamma}), color~{weight_col} bins, top_n={top_n}")

# --- Importance (structural centrality) map ---
marker_map_emphasized(
    plot_df,
    weight_col="consensus_score",
    out_html="boston_markers_importance_emphasized.html",
    top_n=120,
    max_r=24,
    gamma=2.6,   # stronger emphasis helps because consensus_score is compressed
    n_bins=5
)

# --- Demand (volume) map ---
marker_map_emphasized(
    plot_df,
    weight_col="out_strength",
    out_html="boston_markers_demand_emphasized.html",
    top_n=120,
    max_r=24,
    gamma=1.8,   # demand usually already has more spread
    n_bins=5
)


Saved: boston_markers_importance_emphasized.html  | size~consensus_score (gamma=2.6), color~consensus_score bins, top_n=120
Saved: boston_markers_demand_emphasized.html  | size~out_strength (gamma=1.8), color~out_strength bins, top_n=120


In [11]:
import numpy as np
import pandas as pd
import folium

def add_legend(m, title, labels, colors):
    items = "".join(
        f"<div style='display:flex; align-items:center; margin:6px 0;'>"
        f"<span style='width:20px; height:20px; background:{c}; display:inline-block; "
        f"margin-right:12px; border:1px solid #666;'></span>"
        f"<span style='font-size:16px; line-height:1.2;'>{lab}</span></div>"
        for lab, c in zip(labels, colors)
    )
    html = f"""
    <div style="
        position: fixed; bottom: 25px; left: 25px; z-index: 9999;
        width: 300px;
        background: rgba(255,255,255,0.94); padding: 16px 18px;
        border: 1px solid #999; border-radius: 12px;
        box-shadow: 0 2px 10px rgba(0,0,0,0.25);
        ">
        <div style="font-size:18px; font-weight:700; margin-bottom:12px;">{title}</div>
        {items}
    </div>
    """
    m.get_root().html.add_child(folium.Element(html))

def marker_map_emphasized(
    df_in,
    weight_col,
    out_html,
    top_n=120,
    min_r=3,
    max_r=22,
    gamma=2.0,         # >1 emphasizes top stations more
    n_bins=5,          # quantile bins for color
    show_edges=False
):
    d = df_in.dropna(subset=["lat", "lon", weight_col]).copy()
    d[weight_col] = pd.to_numeric(d[weight_col], errors="coerce")
    d = d.dropna(subset=[weight_col])

    d = d.sort_values(weight_col, ascending=False).head(top_n).copy()

    # ----- Size scaling -----
    w = d[weight_col].astype(float).values
    # robust range
    lo, hi = np.quantile(w, 0.05), np.quantile(w, 0.95)
    x = (np.clip(w, lo, hi) - lo) / (hi - lo + 1e-12)  # [0,1]
    x = np.power(x, gamma)                              # emphasize top
    radii = min_r + x * (max_r - min_r)

    # ----- Color bins (quantiles) -----
    qs = np.quantile(w, np.linspace(0, 1, n_bins + 1))
    qs = np.unique(qs)
    if len(qs) < 3:
        qs = np.array([w.min(), np.median(w), w.max()])

    colors = ["#cfe8ff", "#9ecae1", "#6baed6", "#3182bd", "#08519c"]
    colors = colors[-(len(qs)-1):]

    def color_for(val):
        idx = np.searchsorted(qs, val, side="right") - 1
        idx = max(0, min(idx, len(qs) - 2))
        return colors[idx]

    labels = []
    for i in range(len(qs)-1):
        labels.append(f"{qs[i]:.3f} – {qs[i+1]:.3f}")

    # ----- Build map -----
    center_lat, center_lon = float(d["lat"].mean()), float(d["lon"].mean())
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="CartoDB positron")

    for (_, row), r in zip(d.iterrows(), radii):
        nm = row.get("gbfs_station_name")
        if pd.isna(nm) or nm is None:
            nm = row.get("station_name", row.get("station_id"))

        val = float(row[weight_col])
        tooltip = (
            f"{nm}<br>"
            f"{weight_col}={val:.3f}<br>"
            f"pagerank={float(row.get('pagerank', 0)):.3f}, betweenness={float(row.get('betweenness', 0)):.3f}<br>"
            f"out_strength={float(row.get('out_strength', 0)):.3f}, in_strength={float(row.get('in_strength', 0)):.3f}"
        )

        folium.CircleMarker(
            location=[float(row["lat"]), float(row["lon"])],
            radius=float(r),
            color="#333333",
            weight=1,
            fill=True,
            fill_color=color_for(val),
            fill_opacity=0.78,
            opacity=0.9,
            tooltip=folium.Tooltip(tooltip, sticky=True),
        ).add_to(m)

    add_legend(
        m,
        title=f"{weight_col} bins (Top {top_n})",
        labels=labels,
        colors=colors
    )

    m.save(out_html)
    print(f"Saved: {out_html}  | size~{weight_col} (gamma={gamma}), color~{weight_col} bins, top_n={top_n}")

# --- Importance (structural centrality) map ---
marker_map_emphasized(
    plot_df,
    weight_col="consensus_score",
    out_html="boston_markers_importance_emphasized.html",
    top_n=120,
    max_r=24,
    gamma=2.6,
    n_bins=5
)

# --- Demand (volume) map ---
marker_map_emphasized(
    plot_df,
    weight_col="out_strength",
    out_html="boston_markers_demand_emphasized.html",
    top_n=120,
    max_r=24,
    gamma=1.8,
    n_bins=5
)


Saved: boston_markers_importance_emphasized.html  | size~consensus_score (gamma=2.6), color~consensus_score bins, top_n=120
Saved: boston_markers_demand_emphasized.html  | size~out_strength (gamma=1.8), color~out_strength bins, top_n=120
